In [167]:
from datetime import datetime
import pandas as pd
import numpy as np
import requests
import json
from tqdm import tqdm
from time import sleep
# Para trabajar con archivos dotenv y los tokens
# -----------------------------------------------------------------------
import os
import dotenv
dotenv.load_dotenv()



True

In [168]:
# Invocamos de forma segura la contraseña de rapidapi
key = os.getenv("token")

In [169]:
url = "https://tripadvisor-scraper.p.rapidapi.com/hotels/list"

querystring = {"query":"Barcelona", "limit":"50", "adults": "2"}

headers = {
	"x-rapidapi-key": key,
	"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

respuesta=response.json()

In [170]:
with open('hoteleles_barcelona.json', 'w') as archivo:
    json.dump(respuesta, archivo, indent=4)

In [171]:
respuesta

{'link': 'https://www.tripadvisor.com/Hotels-g187497-Barcelona_Catalonia-Hotels.html',
 'total_pages': 73,
 'current_page': 1,
 'total_items_count': 2185,
 'items_count': 30,
 'results': [{'id': 2545026,
   'name': 'Andante Hotel',
   'link': 'https://www.tripadvisor.com/Hotel_Review-g187497-d2545026-Reviews-Andante_Hotel-Barcelona_Catalonia.html',
   'reviews': 2817,
   'rating': 4.5,
   'price_range_usd': {'min': 150, 'max': 241},
   'phone': '+34 934 41 25 45',
   'address': 'Av. Drassanes, 23-25 Rambla Del Raval, 08001 Barcelona Spain',
   'detailed_address': {'street': 'Av. Drassanes, 23-25, Rambla Del Raval',
    'city': 'Barcelona',
    'postal_code': '08001',
    'state': None,
    'country': 'Spain'},
   'ranking': {'current_rank': 91, 'total': 512},
   'featured_image': None,
   'latitude': 41.37648,
   'longitude': 2.172685,
   'amenities': ['Free Wifi', 'Pool', 'Outdoor pool', 'Fitness center'],
   'providers': []},
  {'id': 3533952,
   'name': 'Travelodge Barcelona Pobleno

In [172]:
url = "https://tripadvisor-scraper.p.rapidapi.com/hotels/list"

querystring = {"query":"Santiago de Compostela", "limit":"50", "adults": "2"}

headers = {
	"x-rapidapi-key": key,
	"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

respuesta2=response.json()

In [173]:
respuesta2

{'link': 'https://www.tripadvisor.com/Hotels-g187508-Santiago_de_Compostela_Province_of_A_Coruna_Galicia-Hotels.html',
 'total_pages': 11,
 'current_page': 1,
 'total_items_count': 307,
 'items_count': 30,
 'results': [{'id': 584984,
   'name': 'NH Collection Santiago de Compostela',
   'link': 'https://www.tripadvisor.com/Hotel_Review-g187508-d584984-Reviews-NH_Collection_Santiago_de_Compostela-Santiago_de_Compostela_Province_of_A_Coruna_Galici.html',
   'reviews': 2316,
   'rating': 4.5,
   'price_range_usd': {'min': 97, 'max': 177},
   'phone': '+34 981 55 80 70',
   'address': 'Avenida Burgo das Nacions s/n Campus Norte, 15705, Santiago de Compostela Spain',
   'detailed_address': {'street': 'Avenida Burgo das Nacions s/n, Campus Norte',
    'city': None,
    'postal_code': '15705',
    'state': None,
    'country': 'Spain'},
   'ranking': {'current_rank': 19, 'total': 69},
   'featured_image': None,
   'latitude': 42.889473,
   'longitude': -8.543447,
   'amenities': ['Free Wifi',

In [174]:
with open('hoteleles_santiago.json', 'w') as archivo:
    json.dump(respuesta2, archivo, indent=4)

In [175]:
def extraer_informacion_hoteles(respuesta_json):
    """
    Extrae información relevante sobre hoteles a partir de una respuesta JSON de un servicio de búsqueda de hoteles.

    La función itera sobre los resultados del JSON y extrae información clave sobre cada hotel, 
    incluyendo el nombre, número de reseñas, rating, rango de precios, ciudad, servicios ofrecidos y un enlace al hotel.
    Calcula además el precio medio por noche basado en el rango de precios.

    Args:
        respuesta_json (dict): Un diccionario que representa la respuesta en formato JSON de un servicio de búsqueda de hoteles.

    Returns:
        list[dict]: Una lista de diccionarios, donde cada diccionario contiene la información de un hotel con las siguientes claves:
            - nombre: El nombre del hotel.
            - numero_reviews: Número de reseñas del hotel.
            - rating: Rating del hotel.
            - precio_rango_min: El precio mínimo por noche en USD.
            - precio_rango_max: El precio máximo por noche en USD.
            - precio_medio_por noche: Precio medio por noche calculado a partir del rango de precios.
            - ciudad: Ciudad donde se encuentra el hotel.
            - servicios: Lista de servicios que ofrece el hotel.
            - link: URL que enlaza a la página del hotel.
    """
    lista_diccionarios = []
    res = respuesta_json
    for i in range(0, len(res.get("results", []))):

        nombre = res["results"][i].get("name", "N/A")
        numero_reviews = res["results"][i].get("reviews", 0)
        rating = res["results"][i].get("rating", "N/A")
        
        rango_precios = res["results"][i].get("price_range_usd", {})
        precio_rango_min = rango_precios.get("min", 0)
        precio_rango_max = rango_precios.get("max", 0)
        precio_medio = (precio_rango_max + precio_rango_min) / 2 if (precio_rango_max + precio_rango_min) > 0 else 0
        
        ciudad = res["results"][i].get("detailed_address", {}).get("city", "N/A")
        servicios = res["results"][i].get("amenities", [])
        link = res["results"][i].get("link", "N/A")

        diccionario_hotel= {"nombre":nombre,
                "numero_reviews":numero_reviews,
                "rating": rating,
                "precio_rango_min":precio_rango_min,
                "precio_rango_max":precio_rango_max,
                "precio_medio_por noche":precio_medio,
                "ciudad":ciudad,
                "servicios":servicios,
                "link":link
                }
        lista_diccionarios.append(diccionario_hotel)
    return lista_diccionarios

In [176]:
extraer_informacion_hoteles(respuesta)

[{'nombre': 'Andante Hotel',
  'numero_reviews': 2817,
  'rating': 4.5,
  'precio_rango_min': 150,
  'precio_rango_max': 241,
  'precio_medio_por noche': 195.5,
  'ciudad': 'Barcelona',
  'servicios': ['Free Wifi', 'Pool', 'Outdoor pool', 'Fitness center'],
  'link': 'https://www.tripadvisor.com/Hotel_Review-g187497-d2545026-Reviews-Andante_Hotel-Barcelona_Catalonia.html'},
 {'nombre': 'Travelodge Barcelona Poblenou',
  'numero_reviews': 2625,
  'rating': 4,
  'precio_rango_min': 87,
  'precio_rango_max': 157,
  'precio_medio_por noche': 122.0,
  'ciudad': 'Barcelona',
  'servicios': ['Restaurant', 'Bar/Lounge'],
  'link': 'https://www.tripadvisor.com/Hotel_Review-g187497-d3533952-Reviews-Travelodge_Barcelona_Poblenou-Barcelona_Catalonia.html'},
 {'nombre': 'Hotel Astoria',
  'numero_reviews': 1900,
  'rating': 4.5,
  'precio_rango_min': 118,
  'precio_rango_max': 233,
  'precio_medio_por noche': 175.5,
  'ciudad': 'Barcelona',
  'servicios': ['Free Wifi',
   'Pool',
   'Outdoor pool',

In [177]:
extraer_informacion_hoteles(respuesta2)

[{'nombre': 'NH Collection Santiago de Compostela',
  'numero_reviews': 2316,
  'rating': 4.5,
  'precio_rango_min': 97,
  'precio_rango_max': 177,
  'precio_medio_por noche': 137.0,
  'ciudad': None,
  'servicios': ['Free Wifi',
   'Free parking',
   'Pool',
   'Heated pool',
   'Indoor pool'],
  'link': 'https://www.tripadvisor.com/Hotel_Review-g187508-d584984-Reviews-NH_Collection_Santiago_de_Compostela-Santiago_de_Compostela_Province_of_A_Coruna_Galici.html'},
 {'nombre': 'Parador de Santiago De Compostela',
  'numero_reviews': 3147,
  'rating': 4.5,
  'precio_rango_min': 155,
  'precio_rango_max': 346,
  'precio_medio_por noche': 250.5,
  'ciudad': None,
  'servicios': ['Free Wifi', 'Restaurant', 'Room service'],
  'link': 'https://www.tripadvisor.com/Hotel_Review-g187508-d195761-Reviews-Parador_de_Santiago_De_Compostela-Santiago_de_Compostela_Province_of_A_Coruna_Galicia.html'},
 {'nombre': 'Hotel Compostela',
  'numero_reviews': 764,
  'rating': 4,
  'precio_rango_min': 71,
  'p

In [178]:
def crear_df_hoteles(funcion_extraer_informacion_hoteles):
    """
    Crea y devuelve un DataFrame a partir de la información de hoteles obtenida.

    La función recibe una lista de diccionarios generada por la función `extraer_informacion_hoteles`, 
    convierte esa lista en un DataFrame y elimina las columnas "precio_rango_min" y "precio_rango_max",
    manteniendo solo la información relevante para el análisis.

    Args:
        funcion_extraer_informacion_hoteles (list[dict]): Lista de diccionarios con información sobre hoteles, 
        obtenida de la función `extraer_informacion_hoteles`.

    Returns:
        pd.DataFrame: Un DataFrame con los datos de hoteles, excluyendo las columnas "precio_rango_min" y "precio_rango_max".
    """

    df_hotel= pd.DataFrame(funcion_extraer_informacion_hoteles).drop(columns= ["precio_rango_min", "precio_rango_max"])
    pd.set_option('display.max_colwidth', None)

    df_hotel
    return df_hotel

In [179]:
df_hoteles_barcelona=crear_df_hoteles(extraer_informacion_hoteles(respuesta))
df_hoteles_barcelona.head()

,nombre,numero_reviews,rating,precio_medio_por noche,ciudad,servicios,link
0,Andante Hotel,2817,4.5,195.5,Barcelona,"[Free Wifi, Pool, Outdoor pool, Fitness center]",https://www.tripadvisor.com/Hotel_Review-g187497-d2545026-Reviews-Andante_Hotel-Barcelona_Catalonia.html
1,Travelodge Barcelona Poblenou,2625,4.0,122.0,Barcelona,"[Restaurant, Bar/Lounge]",https://www.tripadvisor.com/Hotel_Review-g187497-d3533952-Reviews-Travelodge_Barcelona_Poblenou-Barcelona_Catalonia.html
2,Hotel Astoria,1900,4.5,175.5,Barcelona,"[Free Wifi, Pool, Outdoor pool, Restaurant, Room service]",https://www.tripadvisor.com/Hotel_Review-g187497-d228590-Reviews-Hotel_Astoria-Barcelona_Catalonia.html
3,Majestic Hotel & Spa Barcelona,4259,4.5,446.0,Barcelona,"[Free Wifi, Pool, Outdoor pool, Restaurant, Airport transportation]",https://www.tripadvisor.com/Hotel_Review-g187497-d190616-Reviews-Majestic_Hotel_Spa_Barcelona-Barcelona_Catalonia.html
4,Oriente Atiram Hotel,2480,4.0,188.0,Barcelona,"[Free Wifi, Restaurant, Room service, Bar/Lounge]",https://www.tripadvisor.com/Hotel_Review-g187497-d250390-Reviews-Oriente_Atiram_Hotel-Barcelona_Catalonia.html


In [180]:
df_hoteles_santiago=crear_df_hoteles(extraer_informacion_hoteles(respuesta2))
df_hoteles_santiago.head()

,nombre,numero_reviews,rating,precio_medio_por noche,ciudad,servicios,link
0,NH Collection Santiago de Compostela,2316,4.5,137.0,None,"[Free Wifi, Free parking, Pool, Heated pool, Indoor pool]",https://www.tripadvisor.com/Hotel_Review-g187508-d584984-Reviews-NH_Collection_Santiago_de_Compostela-Santiago_de_Compostela_Province_of_A_Coruna_Galici.html
1,Parador de Santiago De Compostela,3147,4.5,250.5,None,"[Free Wifi, Restaurant, Room service]",https://www.tripadvisor.com/Hotel_Review-g187508-d195761-Reviews-Parador_de_Santiago_De_Compostela-Santiago_de_Compostela_Province_of_A_Coruna_Galicia.html
2,Hotel Compostela,764,4.0,124.0,None,"[Free Wifi, Room service, Bar/Lounge]",https://www.tripadvisor.com/Hotel_Review-g187508-d248531-Reviews-Hotel_Compostela-Santiago_de_Compostela_Province_of_A_Coruna_Galicia.html
3,Hotel Santiago Plaza Affiliated by Melia,940,4.0,118.0,None,"[Free Wifi, Restaurant, Room service, Bar/Lounge]",https://www.tripadvisor.com/Hotel_Review-g187508-d234587-Reviews-Hotel_Santiago_Plaza_Affiliated_by_Melia-Santiago_de_Compostela_Province_of_A_Coruna_Ga.html
4,Hotel Congreso,922,3.5,66.5,None,"[Free Wifi, Free parking, Pool, Outdoor pool, Restaurant]",https://www.tripadvisor.com/Hotel_Review-g187508-d232804-Reviews-Hotel_Congreso-Santiago_de_Compostela_Province_of_A_Coruna_Galicia.html


Veamos si es necesario limpiar los dataframes que acabamos de obtener

In [181]:
df_hoteles_barcelona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nombre                  30 non-null     object 
 1   numero_reviews          30 non-null     int64  
 2   rating                  30 non-null     float64
 3   precio_medio_por noche  30 non-null     float64
 4   ciudad                  30 non-null     object 
 5   servicios               30 non-null     object 
 6   link                    30 non-null     object 
dtypes: float64(2), int64(1), object(4)
memory usage: 1.8+ KB


In [182]:
df_hoteles_santiago.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nombre                  30 non-null     object 
 1   numero_reviews          30 non-null     int64  
 2   rating                  30 non-null     float64
 3   precio_medio_por noche  30 non-null     float64
 4   ciudad                  1 non-null      object 
 5   servicios               30 non-null     object 
 6   link                    30 non-null     object 
dtypes: float64(2), int64(1), object(4)
memory usage: 1.8+ KB


Observamos que los tipos de datos son coherentes y manejables para el análisis en ambos dataframes. En cuanto a valores nulos, el dataframe de barcelona, no tiene ninguno, mientras que el de santiago los acumula en la columna ciudad. Esto es fácilmente solucionable, rellenando nosotros esos valores pues sabems que la ciudad es Santiago de Compostela.

In [183]:
df_hoteles_santiago['ciudad'] = 'Santiago de Compostela'
df_hoteles_santiago.head()

,nombre,numero_reviews,rating,precio_medio_por noche,ciudad,servicios,link
0,NH Collection Santiago de Compostela,2316,4.5,137.0,Santiago de Compostela,"[Free Wifi, Free parking, Pool, Heated pool, Indoor pool]",https://www.tripadvisor.com/Hotel_Review-g187508-d584984-Reviews-NH_Collection_Santiago_de_Compostela-Santiago_de_Compostela_Province_of_A_Coruna_Galici.html
1,Parador de Santiago De Compostela,3147,4.5,250.5,Santiago de Compostela,"[Free Wifi, Restaurant, Room service]",https://www.tripadvisor.com/Hotel_Review-g187508-d195761-Reviews-Parador_de_Santiago_De_Compostela-Santiago_de_Compostela_Province_of_A_Coruna_Galicia.html
2,Hotel Compostela,764,4.0,124.0,Santiago de Compostela,"[Free Wifi, Room service, Bar/Lounge]",https://www.tripadvisor.com/Hotel_Review-g187508-d248531-Reviews-Hotel_Compostela-Santiago_de_Compostela_Province_of_A_Coruna_Galicia.html
3,Hotel Santiago Plaza Affiliated by Melia,940,4.0,118.0,Santiago de Compostela,"[Free Wifi, Restaurant, Room service, Bar/Lounge]",https://www.tripadvisor.com/Hotel_Review-g187508-d234587-Reviews-Hotel_Santiago_Plaza_Affiliated_by_Melia-Santiago_de_Compostela_Province_of_A_Coruna_Ga.html
4,Hotel Congreso,922,3.5,66.5,Santiago de Compostela,"[Free Wifi, Free parking, Pool, Outdoor pool, Restaurant]",https://www.tripadvisor.com/Hotel_Review-g187508-d232804-Reviews-Hotel_Congreso-Santiago_de_Compostela_Province_of_A_Coruna_Galicia.html


Comprobemos por último si hay filas duplicadas y en caso de haberlas eliminamos el duplicado

In [184]:
#Excluimos de la comprobación la columna servicios, que da problemas por contener lsitas en cada fila
num_filas_duplicadas = df_hoteles_barcelona.drop(columns=['servicios']).duplicated().value_counts()
num_filas_duplicadas

False    30
Name: count, dtype: int64

In [185]:
# Para eliminar la fila duplicada, no nos vale aplicar directamente drop_duplicates
# por ser "servicios" una columna de listas

df_auxiliar = df_hoteles_barcelona.drop(columns=['servicios'])


filas_no_duplicadas = ~df_auxiliar.duplicated()

# Obtenemos el dataFrame original filtrado para eliminar las filas duplicadas
df_hoteles_barcelona = df_hoteles_barcelona[filas_no_duplicadas].reset_index(drop=True)

In [186]:
df_hoteles_barcelona.drop(columns=['servicios']).duplicated().value_counts() #vemos que ya no hay duplicados

False    30
Name: count, dtype: int64

In [187]:
# Mismo proceso para el DataFrame de Santiago de Compostela
num_filas_duplicadas = df_hoteles_barcelona.drop(columns=['servicios']).duplicated().value_counts()
num_filas_duplicadas

False    30
Name: count, dtype: int64

En este caso, el de Santiago no tiene duplicados, por lo que termina aquí la limpieza de ambos DataFrames

In [188]:
df_hoteles_barcelona.to_csv("datos/hoteles_barcelona.csv", index = None)
df_hoteles_santiago.to_csv("datos/hoteles_santiago.csv", index = None)

**Vuelos con Skyscraper**

In [189]:

url = "https://sky-scrapper.p.rapidapi.com/api/v2/flights/searchFlights"

querystring = {
    "originSkyId": "MAD",  # Código para Madrid
    "destinationSkyId": "SCQ",  # Código para Santiago de Compostela
    "originEntityId": "95565077",  # Id de Madrid
    "destinationEntityId": "95565108",  # Id de Santiago de Compostela
    "date": "2024-11-22",
    "returnDate":"2024-11-24",
    "cabinClass": "economy",
    "adults": "2",
    "sortBy": "best",
    "limit":50,
    "currency": "EUR",
}

headers = {
    "x-rapidapi-key": key,
    "x-rapidapi-host": "sky-scrapper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

vuelo_santiago = response.json()


In [190]:
vuelo_santiago

{'status': True,
 'timestamp': 1729455997511,
 'sessionId': 'b58aeab4-c0a2-47d9-93cf-49242ceb48eb',
 'data': {'context': {'status': 'incomplete',
   'sessionId': 'ClQIARJQCk4KJGI1OGFlYWI0LWMwYTItNDdkOS05M2NmLTQ5MjQyY2ViNDhlYhACGiRiMTFlNzE3ZS0xNWJhLTQ2NDItOGM0NS0xNzNlMjU0MmEzMjMSKHVzc180NGJmMDNiMC00MDhjLTQxNTAtOTI1OC00NmYyNTEzZDg1Yjk=',
   'totalResults': 10},
  'itineraries': [{'id': '13870-2411221325--32221,-31685-1-16142-2411230055|16142-2411240615--31915-1-13870-2411241400',
    'price': {'raw': 216.72,
     'formatted': '217 €',
     'pricingOptionId': 'SFZFAGXFZ3oG'},
    'legs': [{'id': '13870-2411221325--32221,-31685-1-16142-2411230055',
      'origin': {'id': 'MAD',
       'entityId': '95565077',
       'name': 'Madrid',
       'displayCode': 'MAD',
       'city': 'Madrid',
       'country': 'Spain',
       'isHighlighted': False},
      'destination': {'id': 'SCQ',
       'entityId': '95565108',
       'name': 'Santiago de Compostela',
       'displayCode': 'SCQ',
       'city

In [191]:
with open('santiago.json', 'w') as archivo:
     json.dump(vuelo_santiago, archivo, indent=4)

In [192]:

url = "https://sky-scrapper.p.rapidapi.com/api/v2/flights/searchFlights"

querystring = {
    "originSkyId": "MAD",  # Código para Madrid
    "destinationSkyId": "BCN",  # Código para Barcelona
    "originEntityId": "95565077",  # Id de Madrid
    "destinationEntityId": "95565085",  # Id de Barcelona
    "date": "2024-11-22",
    "returnDate":"2024-11-24",
    "cabinClass": "economy",
    "adults": "2",
    "sortBy": "best",
    "limit":50,
    "currency": "EUR",
}

headers = {
    "x-rapidapi-key": key,
    "x-rapidapi-host": "sky-scrapper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

vuelo_barcelona = response.json()


In [193]:
vuelo_barcelona

{'status': True,
 'timestamp': 1729456000862,
 'sessionId': 'f8499880-397c-453c-8b92-babed867a626',
 'data': {'context': {'status': 'incomplete',
   'sessionId': 'ClQIARJQCk4KJGY4NDk5ODgwLTM5N2MtNDUzYy04YjkyLWJhYmVkODY3YTYyNhACGiRhODFhMjI1MC02NjcwLTQ4MGItYmM4NS02MzY2ODg0OTIyNWESKHVzc19hNTRjYTQ4Ni00MzUyLTRhZTctOGJjZS1mODNjMDAxNWI5MTE=',
   'totalResults': 10},
  'itineraries': [{'id': '13870-2411221600--32356-1-9772-2411222105|9772-2411241535--31685,-31915-1-13870-2411242205',
    'price': {'raw': 445.68,
     'formatted': '446 €',
     'pricingOptionId': 'ixuWX8zMEzHd'},
    'legs': [{'id': '13870-2411221600--32356-1-9772-2411222105',
      'origin': {'id': 'MAD',
       'entityId': '95565077',
       'name': 'Madrid',
       'displayCode': 'MAD',
       'city': 'Madrid',
       'country': 'Spain',
       'isHighlighted': False},
      'destination': {'id': 'BCN',
       'entityId': '95565085',
       'name': 'Barcelona',
       'displayCode': 'BCN',
       'city': 'Barcelona',
       

In [194]:
with open('barcelona.json', 'w') as archivo:
     json.dump(vuelo_barcelona, archivo, indent=4)

In [195]:
def extraer_informacion_vuelos(respuesta_json):
    """
    Extrae información relevante sobre vuelos a partir de una respuesta JSON de un servicio de itinerarios de vuelos.

    La función itera sobre los vuelos y sus tramos (piernas) en el JSON de respuesta, extrayendo información
    como el precio, aeropuertos de origen y destino, tiempos de salida y llegada, duración del vuelo, escalas y aerolíneas.
    Luego organiza esta información en una lista de diccionarios.

    Args:
        respuesta_json (dict): Un diccionario que representa la respuesta en formato JSON de un servicio de vuelos.

    Returns:
        list[dict]: Una lista de diccionarios, donde cada diccionario contiene la información de un tramo de vuelo con las siguientes claves:
            - id_origen: ID del aeropuerto de origen.
            - ciudad_origen: Nombre de la ciudad de origen.
            - id_destino: ID del aeropuerto de destino.
            - ciudad_destino: Nombre de la ciudad de destino.
            - salida: Hora y fecha de salida del vuelo.
            - llegada: Hora y fecha de llegada del vuelo.
            - duracion(min): Duración del vuelo en minutos.
            - escalas: Número de escalas del vuelo.
            - aerolinea: Nombre de la aerolínea que opera el vuelo.
            - precio: Precio del vuelo (formato raw).
    """
    lista_diccionarios = []
    res = respuesta_json.get("data", {})

    for vuelo in range(0, len(res.get("itineraries", []))): 
        precio = res["itineraries"][vuelo].get("price", {}).get("raw", 0)

        for i in range(0, 2):
            id_origen = res["itineraries"][vuelo]["legs"][i].get("origin", {}).get("id", "N/A")
            origen = res["itineraries"][vuelo]["legs"][i].get("origin", {}).get("city", "N/A")

            id_destino = res["itineraries"][vuelo]["legs"][i].get("destination", {}).get("id", "N/A")
            destino = res["itineraries"][vuelo]["legs"][i].get("destination", {}).get("city", "N/A")

            duracion_minutos = res["itineraries"][vuelo]["legs"][i].get("durationInMinutes", 0)
            escalas = res["itineraries"][vuelo]["legs"][i].get("stopCount", 0)

            salida = res["itineraries"][vuelo]["legs"][i].get("departure", "N/A")
            llegada = res["itineraries"][vuelo]["legs"][i].get("arrival", "N/A")

            aerolinea = res["itineraries"][vuelo]["legs"][i].get("carriers", {}).get("marketing", [{}])[0].get("name", "N/A")

            diccionario_vuelos = {
                "id_origen": id_origen,
                "ciudad_origen": origen,
                "id_destino": id_destino,
                "ciudad_destino": destino,
                "salida": salida,
                "llegada": llegada,
                "duracion(min)": duracion_minutos,
                "escalas": escalas,
                "aerolinea": aerolinea,
                "precio": precio
            }
            lista_diccionarios.append(diccionario_vuelos)
    
    return lista_diccionarios
 

In [196]:
extraer_informacion_vuelos(vuelo_santiago)

[{'id_origen': 'MAD',
  'ciudad_origen': 'Madrid',
  'id_destino': 'SCQ',
  'ciudad_destino': 'Santiago de Compostela',
  'salida': '2024-11-22T13:25:00',
  'llegada': '2024-11-23T00:55:00',
  'duracion(min)': 690,
  'escalas': 1,
  'aerolinea': 'Iberia Express',
  'precio': 216.72},
 {'id_origen': 'SCQ',
  'ciudad_origen': 'Santiago de Compostela',
  'id_destino': 'MAD',
  'ciudad_destino': 'Madrid',
  'salida': '2024-11-24T06:15:00',
  'llegada': '2024-11-24T14:00:00',
  'duracion(min)': 465,
  'escalas': 1,
  'aerolinea': 'Ryanair',
  'precio': 216.72},
 {'id_origen': 'MAD',
  'ciudad_origen': 'Madrid',
  'id_destino': 'SCQ',
  'ciudad_destino': 'Santiago de Compostela',
  'salida': '2024-11-22T07:20:00',
  'llegada': '2024-11-22T23:55:00',
  'duracion(min)': 995,
  'escalas': 1,
  'aerolinea': 'Iberia Express',
  'precio': 200.84},
 {'id_origen': 'SCQ',
  'ciudad_origen': 'Santiago de Compostela',
  'id_destino': 'MAD',
  'ciudad_destino': 'Madrid',
  'salida': '2024-11-24T06:15:00

In [197]:
extraer_informacion_vuelos(vuelo_barcelona)

[{'id_origen': 'MAD',
  'ciudad_origen': 'Madrid',
  'id_destino': 'BCN',
  'ciudad_destino': 'Barcelona',
  'salida': '2024-11-22T16:00:00',
  'llegada': '2024-11-22T21:05:00',
  'duracion(min)': 305,
  'escalas': 1,
  'aerolinea': 'easyJet',
  'precio': 445.68},
 {'id_origen': 'BCN',
  'ciudad_origen': 'Barcelona',
  'id_destino': 'MAD',
  'ciudad_destino': 'Madrid',
  'salida': '2024-11-24T15:35:00',
  'llegada': '2024-11-24T22:05:00',
  'duracion(min)': 390,
  'escalas': 1,
  'aerolinea': 'Vueling Airlines',
  'precio': 445.68},
 {'id_origen': 'MAD',
  'ciudad_origen': 'Madrid',
  'id_destino': 'BCN',
  'ciudad_destino': 'Barcelona',
  'salida': '2024-11-22T10:30:00',
  'llegada': '2024-11-22T16:35:00',
  'duracion(min)': 365,
  'escalas': 1,
  'aerolinea': 'Wizz Air Malta',
  'precio': 440.68},
 {'id_origen': 'BCN',
  'ciudad_origen': 'Barcelona',
  'id_destino': 'MAD',
  'ciudad_destino': 'Madrid',
  'salida': '2024-11-24T15:35:00',
  'llegada': '2024-11-24T22:05:00',
  'duracion

In [198]:
def crear_df_vuelos(funcion_informacion_vuelos):
    """
    Crea y devuelve un DataFrame a partir de la información de vuelos obtenida.

    La función recibe una lista de diccionarios generada por la función `extraer_informacion_vuelos`, 
    convierte esa lista en un DataFrame y formatea las columnas de "salida" y "llegada" 
    como objetos datetime.

    Args:
        funcion_informacion_vuelos (list[dict]): Lista de diccionarios con información sobre vuelos, 
        obtenida de la función `extraer_informacion_vuelos`.

    Returns:
        pd.DataFrame: Un DataFrame con los datos de vuelos, donde las columnas "salida" y "llegada"
        están formateadas como datetime.
    """

    df_vuelos= pd.DataFrame(funcion_informacion_vuelos)
    df_vuelos["salida"]= pd.to_datetime(df_vuelos["salida"])
    df_vuelos["llegada"]= pd.to_datetime(df_vuelos["llegada"])
    return df_vuelos

In [199]:
santiago=extraer_informacion_vuelos(vuelo_santiago)
df_vuelos_santiago=crear_df_vuelos(santiago)

In [200]:
barcelona=extraer_informacion_vuelos(vuelo_barcelona)
df_vuelos_barcelona=crear_df_vuelos(barcelona)

In [201]:
df_vuelos_santiago.head()

,id_origen,ciudad_origen,id_destino,ciudad_destino,salida,llegada,duracion(min),escalas,aerolinea,precio
0,MAD,Madrid,SCQ,Santiago de Compostela,2024-11-22 13:25:00,2024-11-23 00:55:00,690,1,Iberia Express,216.72
1,SCQ,Santiago de Compostela,MAD,Madrid,2024-11-24 06:15:00,2024-11-24 14:00:00,465,1,Ryanair,216.72
2,MAD,Madrid,SCQ,Santiago de Compostela,2024-11-22 07:20:00,2024-11-22 23:55:00,995,1,Iberia Express,200.84
3,SCQ,Santiago de Compostela,MAD,Madrid,2024-11-24 06:15:00,2024-11-24 14:00:00,465,1,Ryanair,200.84
4,MAD,Madrid,SCQ,Santiago de Compostela,2024-11-22 16:10:00,2024-11-23 00:55:00,525,1,Iberia Express,532.82


In [202]:
df_vuelos_barcelona.head()

,id_origen,ciudad_origen,id_destino,ciudad_destino,salida,llegada,duracion(min),escalas,aerolinea,precio
0,MAD,Madrid,BCN,Barcelona,2024-11-22 16:00:00,2024-11-22 21:05:00,305,1,easyJet,445.68
1,BCN,Barcelona,MAD,Madrid,2024-11-24 15:35:00,2024-11-24 22:05:00,390,1,Vueling Airlines,445.68
2,MAD,Madrid,BCN,Barcelona,2024-11-22 10:30:00,2024-11-22 16:35:00,365,1,Wizz Air Malta,440.68
3,BCN,Barcelona,MAD,Madrid,2024-11-24 15:35:00,2024-11-24 22:05:00,390,1,Vueling Airlines,440.68
4,MAD,Madrid,BCN,Barcelona,2024-11-22 10:30:00,2024-11-22 17:20:00,410,1,Wizz Air Malta,406.68


Veamos si es necesario limpiar los archivos

In [203]:
df_vuelos_santiago.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_origen       20 non-null     object        
 1   ciudad_origen   20 non-null     object        
 2   id_destino      20 non-null     object        
 3   ciudad_destino  20 non-null     object        
 4   salida          20 non-null     datetime64[ns]
 5   llegada         20 non-null     datetime64[ns]
 6   duracion(min)   20 non-null     int64         
 7   escalas         20 non-null     int64         
 8   aerolinea       20 non-null     object        
 9   precio          20 non-null     float64       
dtypes: datetime64[ns](2), float64(1), int64(2), object(5)
memory usage: 1.7+ KB


In [204]:
df_vuelos_barcelona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_origen       20 non-null     object        
 1   ciudad_origen   20 non-null     object        
 2   id_destino      20 non-null     object        
 3   ciudad_destino  20 non-null     object        
 4   salida          20 non-null     datetime64[ns]
 5   llegada         20 non-null     datetime64[ns]
 6   duracion(min)   20 non-null     int64         
 7   escalas         20 non-null     int64         
 8   aerolinea       20 non-null     object        
 9   precio          20 non-null     float64       
dtypes: datetime64[ns](2), float64(1), int64(2), object(5)
memory usage: 1.7+ KB


No hay valores nulos que tratar, veamos que pasa con los duplicados.

In [205]:
df_vuelos_santiago.duplicated().value_counts()

False    20
Name: count, dtype: int64

In [206]:
df_vuelos_barcelona.duplicated().value_counts()

False    20
Name: count, dtype: int64

In [207]:
# Tenemos una fila duplicada en cada Data Frame, las eliminamos
df_vuelos_santiago=df_vuelos_santiago.drop_duplicates()
df_vuelos_barcelona=df_vuelos_barcelona.drop_duplicates()

In [208]:
df_vuelos_barcelona.to_csv("datos/vuelos_barcelona.csv", index = None)
df_vuelos_santiago.to_csv("datos/hoteles_santiago.csv", index = None)